In [23]:
import theano
import theano.tensor as T
import numpy as np

In [46]:
lx, ly = T.dscalars('lx', 'ly')
fseq = T.dvector('fseq')
N = fseq.size
delta_x = lx / N
iseq = T.arange(fseq.size)

In [48]:
dfseq, _ = theano.map(fn=lambda i: T.switch(T.eq(i, 0),
                                            fseq[1]-fseq[0],
                                            fseq[i]-fseq[i-1]) / delta_x,
                      sequences=[iseq])

In [49]:
functional_ithterm = lambda i: delta_x * T.sqrt(0.5*(1+(dfseq[i])**2)/(ly-fseq[i]))

In [50]:
functional_parts, _ = theano.map(fn=lambda k: functional_ithterm(k), sequences=[iseq])
functional = functional_parts.sum()

In [51]:
dseqfcn = theano.function(inputs=[fseq,lx], outputs=dfseq)

In [52]:
dseqfcn(np.array([0.75,0.5,0.25,0.]),1.)

array([-1., -1., -1., -1.])

In [44]:
xxseq = T.dvector()

returnlast = xxseq[ xxseq.size-1]
returnlastfcn = theano.function(inputs=[xxseq], outputs=returnlast)

iiseq = T.arange(xxseq.size)
dxxseq, _ = theano.map(fn=lambda i: T.switch(T.eq(i,0), xxseq[0]-xxseq[1], T.switch(T.eq(i,xxseq.size-1),xxseq[xxseq.size-1]*1000,xxseq[i]-xxseq[i-1])),
                       sequences=[iiseq])
dxxseqfcn = theano.function(inputs=[xxseq], outputs=dxxseq)

In [45]:
dxxseqfcn(np.array([0.,1.2,3.1,1.2,4.5]))

array([ -1.20000000e+00,   1.20000000e+00,   1.90000000e+00,
        -1.90000000e+00,   4.50000000e+03])

TypeError: ('Bad input argument to theano function with name "<ipython-input-23-74c003c1e6ef>:2"  at index 0(0-based)', 'TensorType(int32, scalar) cannot store a value of dtype int64 without risking loss of precision. If you do not mind this loss, you can: 1) explicitly cast your data to int32, or 2) set "allow_input_downcast=True" when calling "function".', array([0, 1]))

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18], dtype=int32)

Prod{acc_dtype=int64}.0